## Scraping by Author Name

In [ ]:
import requests
import json
import pandas as pd
import time

#response = requests.get('https://www.googleapis.com/books/v1/volumes?q=inauthor:Caitlin Kelly')
url = 'https://www.googleapis.com/books/v1/volumes'

author_searchables = pd.read_csv('isbn13_1.csv')
df = pd.DataFrame(columns=['author', 'title', 'datepublished', 'isbn13'])

authors = author_searchables['author'].unique()
print(authors.shape)
i=0
for authorsearch in authors:
    #print(f"Searching books for author {authorsearch}")
    if i<10700:
        i=i+1
        continue
    params = {
        'q': f'inauthor:{authorsearch}', 
        'startIndex': 0 
    }

    all_books = []


    while True:
        response = requests.get(url, params=params)
        data = json.loads(response.content)
        #print(data)
        if 'items' in data:
            all_books.extend(data['items'])

        total_items = data.get('totalItems', 0)
        current_items = len(all_books)

        if current_items >= total_items:
            break

        params['startIndex'] = current_items
    #print("All books = ",len(all_books))
    for book in all_books:
        # Access book details using book['volumeInfo']
            identifiers = book["volumeInfo"].get("industryIdentifiers", [])
            if 'authors' not in book["volumeInfo"] or 'publishedDate' not in book["volumeInfo"] or 'title' not in book["volumeInfo"]:
                continue
            title = book["volumeInfo"]["title"]
            author = book["volumeInfo"]["authors"]
            publishedDate = book["volumeInfo"]["publishedDate"][:4]
            isbn13 = 'None'
            for identifier in identifiers:
                if identifier["type"] == "ISBN_13":
                    isbn13 = (identifier["identifier"])
                    break
            if isbn13 == 'None':
                continue
            new_data = ({'Title': title, 
                    'Author': [author], 
                    'PublishedDate': publishedDate, 
                    'ISBN13': isbn13})
            new_row = pd.DataFrame(new_data)
            df = pd.concat([df, new_row], ignore_index=True)
            df = df.drop_duplicates(subset="ISBN13")
    i+=1
    if i%100 ==0:
        print(i)
        time.sleep(1) #Using 1 sec delay since there is a request limit of 100 per sec for google books api
        df.to_csv('All_isbn13_for_sales_by_Author_7.csv')

# Below code is for testing only

In [ ]:
df.to_csv('Google_Author_crosswalk_adv.csv')

In [ ]:
import pandas as pd
import spacy
import re
from fuzzywuzzy import fuzz
# Load the English language model in spaCy
nlp = spacy.load('en_core_web_sm')

In [ ]:
def get_advances_data():
    
    adv = pd.read_excel('../author_advance_dataset.xlsx', index_col=0)
    adv = adv[~adv['Advance'].isna()]
    deals = {
    'NICE' : (1+49000)/2,
    'VERY NICE' : (50000 + 99000)/2,
    'GOOD' : (100000 + 250000)/2,
    'SIGNIFICANT' : (251000 + 499000)/2,
    'MAJOR' : (500000 + 750000)/2
    }
    adv['Advance'] = adv['Advance'].replace(deals)

    adv.reset_index(inplace=True)
    adv.rename(columns={'index': 'advance_id'}, inplace=True) 
    return adv

def extract_author_name_1(text):
    doc = nlp(text)
    for entity in doc.ents:
        if entity.label_ == 'PERSON' and ("'s" in entity.text or "'" in entity.text):
            return entity.text
    return None


def extract_author_name_2(text):
    doc = nlp(text)
    for entity in doc.ents:
        if entity.label_ == 'PERSON' :#and ("'s" in entity.text or "'" in entity.text):
            return entity.text
    return None

def author_extraction(adv):
    adv['Author_extracted'] = adv['All'].apply(lambda x: extract_author_name_1(x))
    adv.loc[(adv['Author_extracted'].isna()) & ((adv['Author(s)'] == 'NONE') | (adv['Author(s)'] == 'All')), 'Author_extracted'] = adv.loc[(adv['Author_extracted'].isna()) & ((adv['Author(s)'] == 'NONE') | (adv['Author(s)'] == 'All')), 'All'].apply(lambda x: extract_author_name_2(x))
    adv.loc[(adv['Author_extracted'].isna()) & (adv['Author(s)'] != 'NONE'), 'Author_extracted'] = adv.loc[(adv['Author_extracted'].isna()) & (adv['Author(s)'] != 'NONE'), 'Author(s)'] 
    adv['Author_extracted'] = adv['Author_extracted'].str.replace("'s", "")
    adv['Author_extracted'] = adv['Author_extracted'].str.replace("'", "")
    adv['Author_extracted'] = adv['Author_extracted'].str.lower()
    return adv

In [ ]:
adv = pd.read_csv('x1.csv')

In [ ]:
adv = author_extraction(adv)

In [ ]:
adv.head()

In [ ]:
authors = adv['Author_extracted'].unique()


In [ ]:
authors.shape

In [ ]:
import requests
import json
import pandas as pd

url = 'https://www.googleapis.com/books/v1/volumes'
df1 = pd.DataFrame(columns=['Author', 'Title', 'PublishedDate', 'ISBN13'])

params = {
        'q': f'inauthor:Brandy Thomas', 
        'startIndex': 0 
    }

all_books = []


while True:
        response = requests.get(url, params=params)
        data = json.loads(response.content)

        if 'items' in data:
            all_books.extend(data['items'])

        total_items = data.get('totalItems', 0)
        current_items = len(all_books)

        if current_items >= total_items:
            break

        params['startIndex'] = current_items
    #print("All books = ",len(all_books))
for book in all_books:
        # Access book details using book['volumeInfo']
            identifiers = book["volumeInfo"].get("industryIdentifiers", [])
            if 'authors' not in book["volumeInfo"] or 'publishedDate' not in book["volumeInfo"] or 'title' not in book["volumeInfo"]:
                continue
            title = book["volumeInfo"]["title"]
            author = book["volumeInfo"]["authors"]
            publishedDate = book["volumeInfo"]["publishedDate"][:4]
            isbn13 = 'None'
            for identifier in identifiers:
                if identifier["type"] == "ISBN_13":
                    isbn13 = (identifier["identifier"])
                    break
            if isbn13 == 'None':
                continue
            new_data = ({'Title': title, 
                    'Author': [author], 
                    'PublishedDate': publishedDate, 
                    'ISBN13': isbn13})
            new_row = pd.DataFrame(new_data)
            df1 = pd.concat([df1, new_row], ignore_index=True)
            df1 = df1.drop_duplicates(subset="ISBN13")

In [ ]:
type(df1['Author'][0])

In [ ]:
df1['Author'] = df1['Author'].astype(str)

In [ ]:
type(df1['Author'][0])

In [ ]:
df1['Author'][0]